In [1]:
import nltk
nltk.download('stopwords')
!python3 -m spacy download en

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [2]:
!pip install pyLDAvis

     |████████████████████████████████| 1.7MB 5.8MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.2.2-py2.py3-none-any.whl size=135593 sha256=1969cd7fca6e9a4a8f7e78763c8929bf809ce97e68b90bd32de0553c49baf96f
  Stored in directory: /root/.cache/pip/wheels/74/df/b6/97234c8446a43be05c9a8687ee0db1f1b5ade5f27729187eae
Successfully built pyLDAvis


In [3]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [4]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
#stop_words.extend(['from', 'subject', 're', 'use'])

In [68]:
df = pd.read_excel("/Usage of Dating Apps (Responses).xlsx")
columns = ['contact', 'reason_for_simultaneous']
df_new = pd.DataFrame()
for col in columns:
  df_new[col] = df[col]

df_new.columns

FileNotFoundError: ignored

In [27]:
df_new = df_new[df_new['reason_for_simultaneous'].notna()]
data = df_new.reason_for_simultaneous.values.tolist()

def sent_to_words(sentences):
  for sentence in sentences:
    yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

data_words = list(sent_to_words(data))
print(data_words[:1])

[['the', 'profile', 'may', 'seem', 'interesting', 'but', 'the', 'person', 'may', 'not', 'end', 'up', 'matching', 'my', 'expectations', 'or', 'may', 'have', 'certain', 'beliefs', 'that', 'are', 'conflicting', 'to', 'mine', 'out', 'of', 'the', 'matches', 'might', 'probably', 'form', 'connection', 'with', 'probably', 'of', 'them', 'conversing', 'with', 'all', 'will', 'be', 'chaotic', 'and', 'won', 'be', 'able', 'to', 'have', 'good', 'conversation', 'with', 'any', 'of', 'them']]


In [ ]:
data

In [32]:
bigram = gensim.models.Phrases(data_words, min_count = 5, threshold = 100)
trigram = gensim.models.Phrases(bigram[data_words], threshold = 100)

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [33]:
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [34]:
print(trigram_mod[bigram_mod[data_words[0]]])

['the', 'profile', 'may', 'seem', 'interesting', 'but', 'the', 'person', 'may', 'not', 'end', 'up', 'matching', 'my', 'expectations', 'or', 'may', 'have', 'certain', 'beliefs', 'that', 'are', 'conflicting', 'to', 'mine', 'out', 'of', 'the', 'matches', 'might', 'probably', 'form', 'connection', 'with', 'probably', 'of', 'them', 'conversing', 'with', 'all', 'will', 'be', 'chaotic', 'and', 'won', 'be', 'able', 'to', 'have', 'good', 'conversation', 'with', 'any', 'of', 'them']


In [45]:
def remove_stopwords(texts):
  return[[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
  return[bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
  return[trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags = ['NOUN', 'ADJ', 'VERB', 'ADV']):
  texts_out = []
  for sent in texts:
    doc = nlp(" ".join(sent))
    texts_out.append([token.lemma_ for token in  doc if token.pos_ in allowed_postags])
  return texts_out

In [46]:
data_words_nostops = remove_stopwords(data_words)
data_words_bigrams = make_bigrams(data_words_nostops)

nlp = spacy.load('en', disable = ['parser', 'ner'])

data_lemmatized = lemmatization(data_words_bigrams, allowed_postags = ['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:1])

[['profile', 'may', 'seem', 'interesting', 'person', 'may', 'end', 'matching', 'expectation', 'may', 'certain', 'belief', 'conflict', 'mine', 'match', 'may', 'probably', 'form', 'connection', 'probably', 'converse', 'chaotic', 'able', 'good', 'conversation']]


In [50]:
id2word = corpora.Dictionary(data_lemmatized)
texts = data_lemmatized

corpus = [id2word.doc2bow(text) for text in texts]

print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 4), (16, 1), (17, 1), (18, 2), (19, 1), (20, 1)]]


In [51]:
id2word[0]

'able'

In [53]:
[[(id2word[ id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('able', 1),
  ('belief', 1),
  ('certain', 1),
  ('chaotic', 1),
  ('conflict', 1),
  ('connection', 1),
  ('conversation', 1),
  ('converse', 1),
  ('end', 1),
  ('expectation', 1),
  ('form', 1),
  ('good', 1),
  ('interesting', 1),
  ('match', 1),
  ('matching', 1),
  ('may', 4),
  ('mine', 1),
  ('person', 1),
  ('probably', 2),
  ('profile', 1),
  ('seem', 1)]]

In [63]:
num_topics = 20
lda_model = gensim.models.ldamodel.LdaModel(corpus = corpus,
                                   id2word = id2word,
                                   num_topics = num_topics,
                                   random_state = 42,
                                   update_every = 1,
                                   chunksize = 100,
                                   passes = 10,
                                   alpha = 'auto',
                                   per_word_topics = True
                                   )

In [64]:
pprint(lda_model.print_topics())
doc_lda = lda_model

[(0,
  '0.115*"interesting" + 0.115*"conversation" + 0.115*"life" + 0.005*"rude" + '
  '0.005*"stranger" + 0.005*"hang" + 0.005*"leave" + 0.005*"option" + '
  '0.005*"difficult" + 0.005*"free"'),
 (1,
  '0.095*"conversation" + 0.095*"interesting" + 0.095*"hold" + 0.095*"come" + '
  '0.095*"man" + 0.005*"may" + 0.005*"probably" + 0.005*"connection" + '
  '0.005*"chaotic" + 0.005*"converse"'),
 (2,
  '0.104*"maybe" + 0.104*"idea" + 0.104*"people" + 0.104*"know" + '
  '0.005*"leave" + 0.005*"rude" + 0.005*"stranger" + 0.005*"hang" + '
  '0.005*"difficult" + 0.005*"option"'),
 (3,
  '0.148*"interesting" + 0.007*"leave" + 0.007*"option" + 0.007*"difficult" + '
  '0.007*"little" + 0.007*"stranger" + 0.007*"hang" + 0.007*"comeback" + '
  '0.007*"hard" + 0.007*"free"'),
 (4,
  '0.156*"feel" + 0.156*"want" + 0.080*"really" + 0.080*"chaotic" + '
  '0.080*"ghostin" + 0.004*"wrong" + 0.004*"conversation" + 0.004*"able" + '
  '0.004*"talk" + 0.004*"many"'),
 (5,
  '0.062*"get" + 0.062*"talk" + 0.04

In [65]:
print('Perplexity = ', lda_model.log_perplexity(corpus))
coherence_model_lda = CoherenceModel(model = lda_model, 
                                     texts = data_lemmatized, 
                                     dictionary=id2word,
                                     coherence = 'c_v'
                                     )

coherence_lda = coherence_model_lda.get_coherence()
print("Coherence score = ", coherence_lda)

Perplexity =  -5.963928300010807
Coherence score =  0.6212457037983294


In [66]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model,
                              corpus,
                              id2word
                              )

vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.184623  0.005626       1        1  27.097183
16     0.204930 -0.064702       2        1  12.470093
15    -0.178794 -0.144728       3        1   9.904610
13     0.085648 -0.121441       4        1   6.080897
8     -0.051684  0.092103       5        1   5.602551
14     0.013955 -0.100830       6        1   4.979729
12    -0.117217 -0.008733       7        1   4.647009
17     0.025331  0.126650       8        1   4.647009
9     -0.007608 -0.068963       9        1   4.160392
19     0.004620  0.035864      10        1   3.698213
4      0.046715  0.088583      11        1   3.438890
1      0.080076 -0.030132      12        1   2.715375
18     0.008525  0.070966      13        1   2.231267
2     -0.053427  0.021710      14        1   2.072332
10     0.005722  0.061497      15        1   1.759354
0      0.065205 -0.031085      16        1   1.613299
7      0.009287  0.027702      17        1   0.875414
3      0.034671  0.004703      18        1   0.875414
6      0.004334  0.017606      19        1   0.565484
11     0.004334  0.017606      20        1   0.565484, topic_info=            Term      Freq     Total Category  logprob  loglift
35        people  8.000000  8.000000  Default   30.000  30.0000
6   conversation  5.000000  5.000000  Default   29.000  29.0000
12   interesting  3.000000  3.000000  Default   28.000  28.0000
36          talk  7.000000  7.000000  Default   27.000  27.0000
47          time  3.000000  3.000000  Default   26.000  26.0000
..           ...       ...       ...      ...      ...      ...
20          seem  0.009456  1.327685  Topic20   -4.804   0.2307
22          feel  0.009456  2.181420  Topic20   -4.804  -0.2659
24          want  0.009456  2.210103  Topic20   -4.804  -0.2789
26          keep  0.009456  2.323273  Topic20   -4.804  -0.3289
27          many  0.009456  5.692245  Topic20   -4.804  -1.2250

[1036 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
0         2  0.518003    able
0         9  0.518003    able
50        1  0.735957  agenda
37        1  0.735679  always
102       3  0.813826  answer
...     ...       ...     ...
24        5  0.452468    want
24       11  0.452468    want
61        1  0.735915    work
101       1  0.735679   would
108       9  0.898869   wrong

[156 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 17, 16, 14, 9, 15, 13, 18, 10, 20, 5, 2, 19, 3, 11, 1, 8, 4, 7, 12])